In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [2]:
!python -V

Python 3.9.12


In [3]:
import pickle
import pandas as pd

In [20]:
import requests
from pathlib import Path

# Define the URL where the model can be downloaded
model_url = 'https://github.com/DataTalksClub/mlops-zoomcamp/raw/refs/heads/main/cohorts/2025/04-deployment/homework/model.bin'
model_path = './model.bin'

# Download the model if it doesn't exist
if not Path(model_path).exists():
    print(f"Downloading model from {model_url}")
    response = requests.get(model_url)
    with open(model_path, 'wb') as f_out:
        f_out.write(response.content)
    print("Model downloaded successfully")

# Load the model
with open(model_path, 'rb') as f_in:
    dv, model = pickle.load(f_in)

/workspaces/mlops-zoomcamp/02-experiment-tracking/.venv/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/workspaces/mlops-zoomcamp/02-experiment-tracking/.venv/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [11]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

## Q1.What's the standard deviation of the predicted duration for this dataset?

In [12]:
std_dev = y_pred.std()
std_dev

np.float64(6.247488852238703)

##  Q2. Preparing the output

In [14]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [17]:
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'predicted_duration': y_pred,
    # 'actual_duration': df['duration'],
    # 'tpep_pickup_datetime': df['tpep_pickup_datetime'],
    # 'tpep_dropoff_datetime': df['tpep_dropoff_datetime'],
    # 'PULocationID': df['PULocationID'],
    # 'DOLocationID': df['DOLocationID'],
    # 'diff': df['duration'] - y_pred
})

# Save the dataframe to a parquet file
output_file = 'predicted_durations.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [19]:
import os

# Get the size of the output file
output_file = 'predicted_durations.parquet'
file_size = os.path.getsize(output_file)

file_size_mb = file_size / (1024 * 1024)  # Convert to megabytes
print(f"File size = {file_size_mb:.03f} Megabyte")

File size = 65.461 Megabyte
